# Training

In [1]:
import sys
sys.path.append('..')

In [2]:
# # Loading useful libraries

import numpy as np
import pandas as pd

import optuna
import os
import pickle

import Demo2

In [3]:
# Selecting "regression" or "classification"

model_type = "classification"

assert(model_type in ["regression", "classification"])

In [4]:
# Loading data

X_train = pd.read_csv(f'../tmp/X_train_{model_type}.csv', index_col='User_ID')
y_train = pd.read_csv(f'../tmp/y_train_{model_type}.csv', index_col='User_ID')['is_BigSpender']

In [5]:
objective = Demo2.hyperparameter_tuning.Objective(X_train=X_train, y_train=y_train)

# Creating a study object and optimize the objective function
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

# Printing the best hyperparameters and corresponding accuracy
print('Best Trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print(f'    {key}: {value}')

  0%|          | 0/100 [00:00<?, ?it/s]

Best Trial:
  Value:  0.4103448275862069
  Params: 
    n_estimators: 842
    num_leaves: 43
    max_depth: 10


In [6]:
study.best_trial

FrozenTrial(number=81, state=1, values=[0.4103448275862069], datetime_start=datetime.datetime(2023, 11, 30, 17, 7, 46, 285021), datetime_complete=datetime.datetime(2023, 11, 30, 17, 7, 47, 219413), params={'n_estimators': 842, 'num_leaves': 43, 'max_depth': 10}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=5, step=1), 'num_leaves': IntDistribution(high=50, log=False, low=10, step=1), 'max_depth': IntDistribution(high=10, log=False, low=2, step=1)}, trial_id=81, value=None)

In [7]:
model = Demo2.training.create_model(study)
model.fit(X_train, y_train)

score_train = Demo2.training.evaluate(model, X_train, y_train)
score_train

0.615983026874116

In [8]:
scores = Demo2.training.evaluate(model, X_train, y_train,cross_val=True)
scores.mean()

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


0.35571260306242636

# Saving model

In [11]:
os.getcwd()

'c:\\Users\\enric\\Documents\\2023 GCP Specialization\\2023_GCP_Specialization_Demo_2-Black_Friday\\notebooks'

In [12]:
try:
   os.makedirs("../tmp")
except FileExistsError:
   # directory already exists
   pass

In [13]:
with open('../tmp/model.pkl', 'wb') as f:
    pickle.dump(model, f)